In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

DEVICE = "cpu"

In [2]:
#################### FUNCTION TO CREATE RBF TRAINING DATASET

pi = torch.pi
start = 0
end = 2 * pi
num_elements = 1000

elements = torch.linspace(start, end, num_elements)

def get_vector(theta, r):
  eps = 10e-5
  term_x_one = (1 + torch.cos(theta)**2)*torch.cos(theta)
  term_x_two_num = (r*(2*((torch.cos(theta))**2) - ((torch.sin(theta))**2) ))*torch.cos(theta)
  denom = torch.sqrt((1+(torch.cos(theta))**2)**2 + (2*torch.cos(theta)*torch.sin(theta))**2 + eps)
  x_term = term_x_one - term_x_two_num/denom

  term_y_one = (1 + (torch.cos(theta)**2))*torch.sin(theta)
  term_y_two_num = (r* (1 + 3*(torch.cos(theta)**2)) )* torch.sin(theta)
  y_term = term_y_one - term_y_two_num/denom

  l_ = torch.sqrt(x_term**2 + y_term**2)
  phi = torch.atan2(y_term,x_term)

  phi = torch.where(phi<0, phi + 2*pi , phi)

  x = (1 + torch.cos(theta)**2)* torch.cos(theta)
  y = (1 + torch.cos(theta)**2)*torch.sin(theta)

  return [x, y, r, phi, x_term, y_term]

In [245]:
################ CREATE DATA SET ################

"""
IMPORTANT : In r, put all radius values that you are planning to use
"""

data = []
r = [0.5,0.4,0.3,0.2,0.1]
for i in range(len(elements)):
  for j in range(len(r)):
    data.append(get_vector(elements[i],r[j]))

data = torch.tensor(data)
train_data = data
print(f"Train data has shape {train_data.shape}")
print("1 -> x of outershape at theta")
print("2 -> y of outershape at theta")
print("3 -> radius of smaller shape")
print("4 -> phi angle of new x and y")
print("5 -> x_term of new point from normal")
print("6 -> y_term of new point from norma")

In [9]:
########################## RBF NETWORK MODEL ####################

class RBFLayer(nn.Module):
    def __init__(self, in_features, out_features, eps, sigma=None):
        super(RBFLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.centers = nn.Parameter(torch.randn(out_features, in_features))
        self.sigma = sigma if sigma else nn.Parameter(torch.ones(1) * 0.5)
        self.eps = eps

    def forward(self, x):
        size = (x.size(0), self.out_features, self.in_features)
        x = x.unsqueeze(1).expand(size)
        c = self.centers.unsqueeze(0).expand(size)
        distances = torch.pow(x - c, 2).sum(-1)
        return torch.exp(-distances / (2 * self.sigma ** 2))
        # return 1/(1+(eps**2)*distances)

class RBFNetwork(nn.Module):
    def __init__(self, in_features, rbf_features, out_features,eps, sigma=None):
        super(RBFNetwork, self).__init__()
        self.rbf = RBFLayer(in_features, rbf_features, eps, sigma)
        self.lol = nn.Linear(rbf_features, rbf_features//2)
        self.linear = nn.Linear(rbf_features//2, out_features)

    def forward(self, x):
        x = self.rbf(x)
        x = self.lol(x)
        x = self.linear(x)
        return x


In [244]:
########################### TRAINING RBF NETWORK ##############################

from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

# Hyperparameters
in_features = 2  # For (phi, r)
rbf_features = 64  # Number of RBF neurons
out_features = 2  # For predicting l'
sigma = 0.4
eps = 1
learning_rate = 1e-3
epochs = 60000

Allowed_length = RBFNetwork(in_features, rbf_features, out_features, eps, sigma).to(DEVICE)
criterion = nn.MSELoss()
optimizer = optim.Adam(Allowed_length.parameters(), lr=learning_rate)


X_train = data[:,2:4].to(DEVICE)
y_train = data[:,4:].to(DEVICE)
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)

# Training loop
for epoch in tqdm(range(epochs)):
    for batch_X, batch_y in train_loader:
      optimizer.zero_grad()
      output = Allowed_length(batch_X)
      loss = criterion(output, batch_y)
      loss.backward()
      optimizer.step()

    if (epoch + 1) % 1000 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')


In [243]:
###################### PLOT FOR CONSTRAINT LEARNED BY RBF NETWORK ###############################

"""
IMPORTANT: We plot constraint for a specified small radius
"""
small_radius = 0.5

fig, ax = plt.subplots(figsize = (10, 6))

# Create an array of theta values from 0 to 2*pi
theta = np.linspace(0, 2 * np.pi, 1000)

# Compute the corresponding r values
r = 1 + np.cos(theta)**2

x = r * np.cos(theta)
y = r * np.sin(theta)

ax.plot(x, y)

theta_ = torch.linspace(0,2*torch.pi, 360)

X = (1+torch.cos(theta_)**2)*torch.cos(theta_)
Y = (1+torch.cos(theta_)**2)*torch.sin(theta_)

r = torch.tensor([small_radius]*len(theta_))

input = torch.stack((r, theta_),dim=1)
Allowed_length = Allowed_length.to("cpu")
xy = (Allowed_length.forward(input)).squeeze(1)
print(xy.shape)
x_ = xy[:,0].detach().numpy()
y_ = xy[:,1].detach().numpy()

ax.plot(x_,y_, markersize=1)
ax.scatter(0.4244, -0.5519, s=25, color="red")

eps = 10e-5
term_x_one = (1 + torch.cos(theta_)**2)*torch.cos(theta_)
term_x_two_num = (small_radius*(2*((torch.cos(theta_))**2) - ((torch.sin(theta_))**2) ))*torch.cos(theta_)
denom = torch.sqrt((1+(torch.cos(theta_))**2)**2 + (2*torch.cos(theta_)*torch.sin(theta_))**2 + eps)
x_term = term_x_one - term_x_two_num/denom

term_y_one = (1 + (torch.cos(theta_)**2))*torch.sin(theta_)
term_y_two_num = (small_radius* (1 + 3*(torch.cos(theta_)**2)) )* torch.sin(theta_)
y_term = term_y_one - term_y_two_num/denom

ax.plot(x_term,y_term)

In [217]:
########### MODEL CLASSES ###############

class ConstraintLayer(nn.Module):
    """
    ConstraintLayer is used to rotate and scale the centers given by encoder such that the inner shapes fit inside outer shape

    Methods:
        __init__: Defines variables.
        forward: Does forward pass through the layer
    """
    def __init__(self, R, maxnorm, input_shape):
        """
        Initializes ConstraintLayer needed variables

        Args:
            R (torch.tensor): Circumradius of the polygon
            maxnorm (torch.tensor) : Vector of all inner radii
            input_shape (torch.tensor) : Shape of input in forward pass

        Variables:
            weight(torch.tensor) : Parameter used for allowing rotation of center location
            a (torch.tensor) : Parameter used for allowing scaling of the center location
        """
        super(ConstraintLayer, self).__init__()
        self.maxnorm = maxnorm
        self.R = R
        self.weight = torch.nn.Parameter(data=torch.randn(input_shape),requires_grad=True)
        self.sigmoid = nn.Sigmoid()
        self.a = torch.nn.Parameter(data=torch.randn(input_shape[0],1)-0.75, requires_grad=True)
        
        
    def forward(self, inputs):
        """
        Forward pass through the constraint layer

        Args:
            inputs (torch.tensor) : Input to the forward pass
        Returns:
            output (torch.tensor) : Output through the layer
        """
        rotation_vector = self.weight / torch.norm(self.weight, dim=1, p=2, keepdim=True).detach()
        inputs = inputs / torch.norm(inputs, dim=1, p=2, keepdim=True).detach()
        mul = torch.cat(((rotation_vector[:,0]*inputs[:,0]-rotation_vector[:,1]*inputs[:,1]).unsqueeze(-1), (rotation_vector[:,1]*inputs[:,0]+rotation_vector[:,0]*inputs[:,1]).unsqueeze(-1)), -1)
        theta = torch.atan2(mul[:, 1], mul[:, 0])
        theta = torch.where(theta < 0, theta + 2 * torch.pi, theta)
        pi = torch.pi
        input = torch.stack((self.maxnorm, theta.unsqueeze(-1)),dim=1)
        xy = Allowed_length.forward(input.squeeze(-1))
        x = xy[:,0]
        y = xy[:,1]
        allowed_length = torch.sqrt(x**2 + y**2).unsqueeze(-1)
        output = mul * allowed_length.detach() * self.sigmoid(1.5*self.a)
        return output
    
class NoiseLayer(nn.Module):
    """
    NoiseLayer perturb the center

    Methods:
        __init__: Defines variables.
        forward: Does forward pass through the layer
    """

    def __init__(self, noise_radius):
        """
        Initializes NoiseLayer needed variables

        Args:
            noiseradius (torch.tensor) : Vector of all inner radii

        Variables:
            alpha (torch.tensor) : Initialize alpha parameter for beta distribution
            beta (torch.tensor) : Initialize beta parameter for beta distribution
        """
        super(NoiseLayer, self).__init__()
        self.noise_radius = noise_radius
        self.alpha = 0.5
        self.beta = 0.5
        
    def beta_distribution_noise(self, num_circles):
        """
        Creating noise from beta distribution

        Args:
            num_circles (torch.tensor) : Total number of inner circles
        Returns:
            noise (torch.tensor) : Noise according to the beta distribution
        """

        x = torch.randn(num_circles, 2)
        beta_distribution = torch.distributions.Beta(self.alpha, self.beta)
        u = beta_distribution.sample((num_circles,2))
        norm = torch.norm(x, dim=1, keepdim=True, p=2)
        noise = x * (u) * self.noise_radius / norm
        return noise
    
    def forward(self, inputs, alpha, beta):
        """
        Forward pass through the constraint layer

        Args:
            inputs (torch.tensor) : Input to the forward pass
            alpha (torch.tensor) : Alpha 
        Returns:
            output (torch.tensor) : Output through the layer
        """
        self.alpha = alpha
        self.beta = beta
        noise = self.beta_distribution_noise(inputs.size(0))
        return inputs + noise
    
class ResidualBlock(nn.Module):
    """
    ResidualBlock

    Methods:
        __init__: Defines variables.
        forward: Does forward pass through the layer
    """
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(True),
            nn.Linear(in_features, in_features)
        )
        
    def forward(self, x):
        return x + self.block(x)
    

class EncoderDecoder(nn.Module):
    """
    Full enocoder-decoder architecure for circle packing

    Methods:
        __init__: Defines variables.
        forward: Does forward pass through the layer
        regions: Gives a detailed diagram highlighting the working of the model
        centers: Gives centers learned by the model
        encoderout: Gives centers learned by the encoder
    """
    def __init__(self, num_circles, larger_radius, smaller_radius, hidden_size=64 , no_of_peturbations=1):
        """
        Initializes NoiseLayer needed variables

        Args:
            num_circles (torch.tensor) : Number of circles to be packed
            larger_radius (torch.tensor) : Outer radius for outer shape circle, Circumradius for outer shape polygon, ~ for RBF trained constraint
            smaller_radius (torch.tensor) : Vector of all inner radii
            hidden_size (torch.tensor) : Hidden size of the model
            no_of_peturbations (torch.tensor) : No of peturbations to be done for one forward pass
        """
        super(EncoderDecoder, self).__init__()
        
        self.main_dim = num_circles
        self.latent_dim = 2
        self.R = larger_radius
        self.r = smaller_radius
        self.mid_dim = hidden_size*2
        self.mid_mid_dim = hidden_size
        self.alpha = 0.5
        self.no_of_peturbations = no_of_peturbations

        self.normalize = ConstraintLayer(self.R, self.r, (num_circles, 2))       
        self.noise = NoiseLayer(self.r)
        
        self.encoder = nn.Sequential(
            nn.Linear(self.main_dim, self.mid_mid_dim),
            nn.BatchNorm1d(self.mid_mid_dim),
            nn.LeakyReLU(),
            nn.Linear(self.mid_mid_dim, self.latent_dim),
            nn.Tanh()
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.latent_dim, 2*self.main_dim),
            nn.BatchNorm1d(2*self.main_dim),
            nn.LeakyReLU(),
            nn.Linear(2*self.main_dim, 2*self.mid_mid_dim),
            nn.BatchNorm1d(2*self.mid_mid_dim),
            nn.LeakyReLU(),
            ResidualBlock(2*self.mid_mid_dim),
            nn.BatchNorm1d(2*self.mid_mid_dim),
            ResidualBlock(2*self.mid_mid_dim),
            nn.BatchNorm1d(2*self.mid_mid_dim),
            ResidualBlock(2*self.mid_mid_dim),
            nn.Linear(2*self.mid_mid_dim, self.main_dim),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x, alpha, beta):
        """
        Forward pass through the model

        Args:
            x (torch.tensor) : Input to the encoder-decoder model, Shape is [num_circles, num_circles]
            alpha (torch.tensor) : Alphas for all peturbation points, Shape is [no_of_peturbations]
            beta (torch.tensor) : Betas for all peturbation points, Shape is [no_of_peturbations]
        """
        encoded = self.encoder(x)
        normal_encoded = self.normalize(encoded)
        noisy_encoded = torch.tensor([])
        for i in range(self.no_of_peturbations):
            noisy_encoded_temp = self.noise(normal_encoded, alpha[i], beta[i])
            noisy_encoded = torch.cat((noisy_encoded, noisy_encoded_temp), dim=0)
        decoded = self.decoder(noisy_encoded)
        return decoded

    def regions(self, x, alpha, beta, colors):
        """
        Gives detailed diagram of the model and its working 

        Args:
            x (torch.tensor) : Input to the encoder-decoder model, Shape is [num_circles, num_circles]
            alpha (torch.tensor) : Alphas for all peturbation points, Shape is [no_of_peturbations]
            beta (torch.tensor) : Betas for all peturbation points, Shape is [no_of_peturbations]
            colors : Different colors for the plotting of circles, Size is [num_circles]
        """
        with torch.no_grad():
            encoded = self.encoder(x)
            normal_encoded = self.normalize(encoded)
            noisy_encoded = torch.tensor([])
            for i in range(self.no_of_peturbations):
                noisy_encoded_temp = self.noise(normal_encoded, alpha[i], beta[i])
                noisy_encoded = torch.cat((noisy_encoded, noisy_encoded_temp), dim=0)
            decoded = self.decoder(noisy_encoded).cpu()
            noisy_encoded = noisy_encoded.cpu()

            # Plotting the data
            for i in range(noisy_encoded.shape[0]):
                label_index = int(np.argmax(decoded[i]))
                plt.scatter(noisy_encoded[i, 0], noisy_encoded[i, 1], color=colors(label_index), label=f'Class {label_index}')
                plt.annotate("guess"+str(label_index), (noisy_encoded[i, 0], noisy_encoded[i, 1]), fontsize=8, ha='center', va='bottom')
            
            plt.xlabel('X-axis')
            plt.ylabel('Y-axis')
            plt.title('2D Coordinates with Different Colors')
            plt.show()
        return 0


    def centers(self, x):
        """
        Outputs centers learned by the model

        Args:
            x (torch.tensor) : Input to the encoder-decoder model, Shape is [num_circles, num_circles]
        Returns:
            centers (torch.tensor) : Gives centers of the circles, Shape is [num_circles, 2]
        """
        centers = self.normalize(self.encoder(x)).cpu().detach().numpy()
        return centers

    def enncoderout(self, x):
        """
        Outputs centers learned by the encoder

        Args:
            x (torch.tensor) : Input to the encoder-decoder model, Shape is [num_circles, num_circles]
        Returns:
            centers (torch.tensor) : Gives centers learned by the encoder, Shape is [num_circles, 2]
        """
        encoderout = self.encoder(x).cpu().detach().numpy()
        return encoderout

In [218]:
def one_hot(a, num_classes):
    return torch.tensor(np.eye(num_classes)[a], dtype=torch.float32)

In [231]:
#################### MODEL COLOR VISUALIZATION PLOTTING CODE ########################

def color_visualization_plot(centres, large_radius, small_radius, colors, save = False, savename=None):
    fig = plt.figure(figsize=(7, 7))
    ax = plt.gca()
    ax.axis('off')
    theta = np.linspace(0, 2 * np.pi, 1000)

    # Compute the corresponding r values
    r = 1 + np.cos(theta)**2

    x = r * np.cos(theta)
    y = r * np.sin(theta)

    ax.plot(x, y, linewidth=3.0, color='black')
    ax.axis('scaled')
    i = 0
    for rad, centre in enumerate(centres):
        (x,y) = centre
        ax.add_patch(plt.Circle(centre, radius = small_radius[i], fill = False, linewidth = 1.0, color = colors(rad)))
        ax.annotate(str(rad), (x, y), (x, y), fontsize=12, ha='center', va='center')
        i += 1
    if save:
        plt.savefig(savename)


In [232]:
#################### BEST MODEL PLOTTING CODE ########################

def plot_circles_final(centres, large_radius, small_radii, save=False, savename=None):
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.axis('off')

    # Plot the center point
    ax.scatter(0, 0, color='tomato')                                                                                                # Center marker

    theta = np.linspace(0, 2 * np.pi, 1000)

    # Compute the corresponding r values
    r = 1 + np.cos(theta)**2

    x = r * np.cos(theta)
    y = r * np.sin(theta)

    # Create a Cartesian plot
    plt.figure(figsize=(8,6))
    ax.plot(x, y, linewidth=3.0, color='black')

    # Plot each inner circle with its respective radius
    for rad, (centre, small_radius) in enumerate(zip(centres, small_radii)):
        # Ensure centre is in the correct format
        centre = np.array(centre)

        # Convert small_radius to a scalar
        small_radius = small_radius.item() if isinstance(small_radius, torch.Tensor) else small_radius

        # Plot inner circle
        ax.add_patch(plt.Circle(centre, radius=small_radius, fill=False, linewidth=1.0, color='dodgerblue'))

    ax.axis('scaled')
    plt.tight_layout()
    fig.patch.set_alpha(0)  # Make the figure background transparent
    ax.set_facecolor((1, 1, 1, 0))  # Make the axes background transparent

    if save:
        plt.savefig('figure.png', bbox_inches='tight')  # Save the figure
    plt.show()

In [233]:
################## TRAINING LOOP PLOT ########################

def plot_circles(centres, large_radius, small_radius, save = False, savename=None):
    fig, ax = plt.subplots(figsize = (3, 3))
    ax.axis('on')
    ax.scatter(0,0, color = 'red')
    # Number of sides of the pentagon
    # num_sides = 4

    theta = np.linspace(0, 2 * np.pi, 1000)

    # Compute the corresponding r values
    r = 1 + np.cos(theta)**2

    x = r * np.cos(theta)
    y = r * np.sin(theta)

    # Create a Cartesian plot
    plt.figure(figsize=(8,6))
    ax.plot(x, y)

    ax.axis('scaled')
    i=0
    for rad, centre in enumerate(centres):
        (x,y) = centre
        #ax.annotate(str(rad+1), (x,y), (x,y), fontsize = 16)
        ax.add_patch(plt.Circle(centre, radius = small_radius[i], fill = False, linewidth = 1.0, color = 'blue'))
        i += 1
    if save:
        plt.savefig(savename)
    plt.show()

def packing_density(centres, r, R = 1, NumSamples = 100000):
    '''
    centres: centres of the spheres [N,2], N: Number of spheres
    r: radius of each circle
    R: radius of enclosing circle
    NumSamples: number of points to be sampled towards density calculation
    '''
    theta = np.random.uniform(0, 2*np.pi, size=NumSamples)

    # Step 2: Calculate r based on the equation r = 1 + cos^2(theta)
    r_max = 1 + np.cos(theta)**2

    # Step 3: Sample random radius values between 0 and r_max
    r_sampled = np.sqrt(np.random.uniform(0, 1, size=NumSamples)) * r_max

    # Step 4: Convert polar coordinates (r_sampled, theta) to Cartesian (x, y)
    x = r_sampled * np.cos(theta)
    y = r_sampled * np.sin(theta)

    #these points are distributed uniformly inside the circle
    coordinates = np.stack([x, y], axis=1)
    centres_reshaped = np.reshape(centres, [1, centres.shape[0], centres.shape[1]])
    coordinates_reshaped = np.reshape(coordinates, [coordinates.shape[0], 1, coordinates.shape[1]])
    distances_from_centres = np.linalg.norm(coordinates_reshaped - centres_reshaped, axis = 2)


    return np.sum(np.any(distances_from_centres<r.T.cpu().numpy(), axis=1))/NumSamples

In [242]:
################# INITIALIZING PARAMETERS ##############
BATCHES = 64
EPOCHS = 10000
DEVICE = DEVICE                                                     # Defined at top
torch.set_default_device(DEVICE)
RUNS = 4
PATIENCE = 50
OUTER_RADIUS = 1
LEARNING_RATE = 0.0005

small_radius = torch.tensor(([0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3],[0.3]))
num_circles = len(small_radius) 
large_radius = torch.ones_like(small_radius) * OUTER_RADIUS         # This is irrelevant for any shape
no_of_peturbations = 5
Allowed_length = Allowed_length.to(DEVICE)                          # RBF Model for getting constraint

################ CREATING DATASET ##################
eye = one_hot(np.arange(0, num_circles), num_circles)
Input = eye.repeat(BATCHES, 1, 1).to(DEVICE)



################## TRAINING LOOP ####################
max_density_runs = [0]*RUNS
run_reached = 0
for run in range(RUNS):
    run_reached += 1
    ############ DEFINE MODEL AND VARIABLES FOR THE RUN #################
    model = EncoderDecoder(num_circles, large_radius, small_radius, no_of_peturbations=no_of_peturbations).to(DEVICE)
    loss = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    alpha = [2]*no_of_peturbations                                                                                  # Initial alpha 
    beta = [8]*no_of_peturbations                                                                                   # Inital beta
    loss_list = []                                                                                                  # Loss list
    max_packing_density = 0                                                                                         # Maximum packing density till now
    curr_packing_density = 0                                                                                        # Current epoch packing density
    stagnation_counter = 0                                                                                          # Stagnation counter for scheduling beta distribution change


    ########### TRAINING MODEL #############
    for epoch in tqdm(range(EPOCHS)):
        epoch_loss = 0

        ########## MAXIMUM ALLOWED ALPHA AND MINIMUM ALLOWED BETA #############
        for i in range(len(beta)):
            if beta[i] < (2.0*((i+1)**(1/no_of_peturbations))):
                beta[i] = (2.0*((i+1)**(1/no_of_peturbations)))
        for i in range(len(alpha)):
            if alpha[i] > (16.0/((i+1)**(1/no_of_peturbations))):
                alpha[i] = 16.0

        for inputs in Input:
            optim.zero_grad()
            outputs = model(inputs, alpha, beta)
            curr_loss = loss(outputs, torch.cat([inputs.argmax(dim=1)]*no_of_peturbations, dim=0)) #inputs.argmax(dim=1))
            curr_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
            optim.step()
            epoch_loss += curr_loss.detach().cpu()
        
        ########## APPEND LOSS ##########
        loss_list.append(epoch_loss)

        ########### SAVE MAXIUMUM PACKING DENSITY YET MODEL ################## 
        centers = model.centers(eye)
        curr_packing_density = packing_density(centers, small_radius, large_radius)
        if max_packing_density < curr_packing_density:
            max_packing_density =  curr_packing_density
            max_density_runs[run] = max_packing_density
            torch.save(model, f'model_{run}.pth')
    

        ########### CALCULATE STAGNATION FOR BETA DISTRIBUTION SCHEDULAR #############
        if (epoch > 0) and curr_packing_density < max_packing_density:
            stagnation_counter += 1
        else:
            stagnation_counter = 0
        
        ########## UPDATE BETA DISTRIBUTION #############
        if stagnation_counter >= PATIENCE:
            for i in range(len(beta)):
                alpha[i]+=0.28/(i+1)
                beta[i]-=0.6/(i+1)
                stagnation_counter = 0
        
        ########## PRINT PACKING ##############
        if epoch % 100 == 0:
            print(f"Epcoh {epoch}") 
            centers = model.centers(eye)
            print(f"Packing Density : {packing_density(centers, small_radius, large_radius)}")
            plot_circles(centers, large_radius, small_radius)
            print("##########################")

In [239]:
########### FINAL PLOTS AND PACKING DENSITY FOR BEST MODELS #################

for run in range(run_reached):
    print("##################################################################################")
    model = torch.load(f'model_{run}.pth')
    centres = model.centers(eye)
    print(f"Packing Density : {packing_density(centres, small_radius, large_radius, NumSamples=100000)}")
    plot_circles_final(centres, large_radius, small_radius, save=True)
    print("##################################################################################")

In [240]:
#################### MODEL WORKING VISUALIZATION PLOT #######################

from matplotlib.colors import ListedColormap, to_rgba_array

"""
IMPORTANT : CHANGE hex_colors to have as many colors as there are circles
"""
hex_colors = [
    '#dcdcdc', '#8b4513', '#708090', '#808000', '#483d8b',
    '#3cb371', '#000080', '#9acd32', '#8b008b', '#48d1cc',
    '#ff4500', '#ffa500', '#ffff00', '#00ff00', '#8a2be2',
    '#00ff7f', '#00bfff', '#0000ff', '#ff00ff', '#1e90ff',
    '#db7093', '#f0e68c', '#ff1493', '#ffa07a', '#ee82ee',
    '#dff993', '#66e68c', '#edc033', '#afbf55', '#ad4512',
    '#b99b95', '#ababab', '#12a800', '#4a1150', '#ffddbb',
    '#b316ff', '#aff551', '#fd5522', '#a22f4d', '#afa552',
    '#ffffff', '#c256aa', '#cc4df5', '#ddd555', '#15a2d3',
]
hex_colors = hex_colors[:num_circles]
# Convert hex colors to RGBA format
rgba_colors = [to_rgba_array(color) for color in hex_colors]

# Create ListedColormap
custom_cmap = ListedColormap(rgba_colors, name='custom_cmap')

for run in range(run_reached):
    print("##################################################################################")
    model = torch.load(f'model_{run}.pth')
    print("VISUALIZATION OF THE PACKING") 
    centers = model.centers(eye)
    colors = custom_cmap
    color_visualization_plot(centers, large_radius, small_radius, colors, save=False)
    model.regions(eye, alpha, beta, colors)
    print("##################################################################################")

In [ ]:
########################################################################### END #################################################################